In [1]:
import numpy as np
import os
import sys
import pandas as pd
import pyreadr
from plotnine import *
from mizani.formatters import percent_format

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-as_g1bz2 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
WHO = {"African":["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi",
    "Cameroon", "Cape Verde", "Central African Republic", 
    "Chad", "Comoros", "Ivory Coast", "Congo, the Democratic Republic of the", 
    "Equatorial Guinea", "Eritrea", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", 
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia", "Madagascar", "Malawi", "Mali", "Mauritania", 
    "Mauritius", "Mozambique", "Namibia", "Niger", "Nigeria", "Republic of the Congo", "Rwanda", 
    "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", 
    "South Africa", "South Sudan", "Eswatini", "Togo", "Uganda", "Tanzania, United Republic of", "Zambia", "Zimbabwe"],
"Americas":["Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", 
            "Bolivia", "Brazil", "Canada", "Chile", "Colombia", 
            "Costa Rica", "Cuba", "Dominica", "Dominican Republic", 
            "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", 
            "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", 
            "Paraguay", "Peru", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", 
            "Suriname", "Trinidad and Tobago", "United States", "Uruguay", "Venezuela"],
"South-East Asian":["Bangladesh","Bhutan","Korea, Republic of","India","Indonesia","Maldives",
                    "Myanmar", "Nepal", "Sri Lanka", "Thailand", "Timor-Leste"],
"European":["Albania","Andorra","Armenia","Austria","Azerbaijan","Belarus","Belgium","Bosnia and Herzegovina",
            "Bulgaria","Croatia","Cyprus","Czechia","Denmark",
            "Estonia","Finland","France","Georgia","Germany","Greece","Hungary",
            "Iceland","Ireland","Israel","Italy","Kazakhstan","Kyrgyzstan","Latvia","Lithuania",
            "Luxembourg","Malta","Moldova, Republic of","Monaco","Montenegro","Netherlands",
            "North Macedonia","Norway","Poland","Portugal","Romania","Russian Federation","San Marino",
            "Serbia","Slovakia","Slovenia","Spain","Sweden","Switzerland","Tajikistan","Turkey", "Liechtenstein",
            "Turkmenistan","Ukraine","United Kingdom","Uzbekistan", "Kosovo"],
"Eastern Mediterranean":["Afghanistan","Bahrain","Djibouti","Egypt","Iran, Islamic Republic of","Iraq","Jordan","Kuwait","Lebanon","Libya","Morocco","Oman","Pakistan","Palestine, State of","Qatar","Saudi Arabia","Somalia","Sudan","Syrian Arab Republic","Tunisia","United Arab Emirates","Yemen"],
"Western Pacific":["Australia","Brunei","Cambodia","China","Cook Islands","Fiji","Japan","Kiribati","Laos","Malaysia","Marshall Islands","Micronesia","Mongolia","Nauru","New Zealand","Niue","Palau","Papua New Guinea","Philippines","Samoa","Singapore","Solomon Islands","South Korea","Tonga","Tuvalu","Vanuatu","Viet Nam"]}   

In [3]:
sids = ['Anguilla' ,'American Samoa' ,'Jamaica' ,'Papua New Guinea' ,'British Virgin Islands' ,
        'Kiribati' ,'Saint Vincent and the Grenadines' ,'Vanuatu' ,'Bahamas' ,
        'Fiji' ,'Puerto Rico' ,'Timor-Leste' ,'Dominican Republic' ,'New Caledonia' ,
        'United States Virgin Islands' ,'Comoros' ,'Aruba' ,'Federated States of Micronesia' ,'Netherlands Antilles' ,
        'Solomon Islands' ,'Dominica' ,'Nauru' ,'Trinidad and Tobago' ,'Cape Verde' ,
        'Belize' ,'Guam' ,'Saint Lucia' ,'Tuvalu' ,'Guyana' ,
        'Northern Mariana Islands' ,'Seychelles' ,'Maldives' ,'Antigua and Barbuda' ,'Cook Islands' ,
        'Montserrat' ,'Samoa' ,'Cuba' ,'Marshall Islands' ,'Suriname' ,
        'Bahrain' ,'Barbados' ,'French Polynesia' ,'Saint Kitts and Nevis' ,'Tonga' ,
        'Grenada' ,'Niue' ,'Sao Tome and Principe' ,'Guinea-Bissau' ,'Haiti' ,'Palau' ,'Singapore' ,'Mauritius']
tier1_countries = ["USA", "CHN"]
tier1_countries_eu = ["Austria", "Belgium", "Bulgaria", "Croatia",  "Cyprus", 
                      "Czechia","Denmark", "Estonia", "Finland", "France","Germany","Greece", 
                      "Hungary", "Ireland", "Italy","Latvia", "Lithuania",
                      "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal","Romania", 
                      "Slovakia", "Slovenia","Spain", "Sweden", "United Kingdom"]
EEAandCO = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", 
            "Czechia","Denmark", "Estonia", "Finland", "France","Germany","Greece", 
            "Hungary", "Iceland","Ireland", "Italy","Latvia", "Liechtenstein", "Lithuania", 
            "Luxembourg", "Malta", "Netherlands", "Norway", "Poland", "Portugal","Romania", 
            "Slovakia", "Slovenia","Spain", "Sweden", "Switzerland", "Turkey", 
            "Albania",  "Bosnia and Herzegovina", "Kosovo", "Montenegro", "North Macedonia", "Serbia", 
            "United Kingdom"]
tier2_countries = ["POL","AUS", "ZAF", "BRA", "IND", "IDN", "DEU","FRA","GBR"] # 

In [4]:
hdi = pd.read_csv("2020_hdi.csv", header = None)
hdi.set_axis(["index", "Country", "score", "HDI"], axis=1, inplace=True)
hdi.replace({"Korea (Republic of)": "Korea, Republic of", "Russian Federation":"Russian Federation",
            'Bolivia (Plurinational State of)': "Bolivia", "Congo (Democratic Republic of the)": 'Congo, the Democratic Republic of the',
            "Tanzania (United Republic of)": "Tanzania, United Republic of",
            "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
            "Moldova (Republic of)": "Moldova, Republic of", 
            "Venezuela (Bolivarian Republic of)": "Venezuela",
            "Côte d'Ivoire": "Cote d'Ivoire"}, inplace = True)

In [5]:
# load data
folder =   "Multiglobal_result/" # "English_result/" #     "EU_result/"  #
df_health = pd.read_pickle(folder + 'df_kwic_health.pkl')
df_climate = pd.read_pickle(folder + 'df_kwic_climate.pkl')
df_intersection = pd.read_csv(folder + 'df_kwic_health_climate.csv')
df_gender = pd.read_csv(folder + 'gender.csv')
df_covid = pd.read_csv(folder + 'covid.csv')

In [6]:
path = "results/" + folder

#os.mkdir(path)

In [ ]:
csr = pd.read_csv('Multiglobal_result/multiglobal_english_all.csv', chunksize = 10000)
if folder == "EU_result/":
    EU = pd.DataFrame()
    for df in csr:
        csr_mask = [True if i in EEAandCO else False for i in df.Country]
        EU = EU.append(df[csr_mask])
    csr = EU
elif folder == "English_result/":
    EN = pd.DataFrame()
    for df in csr:
        EN = EN.append(df[df.English >0])
    csr = EN
else:
    glob = pd.DataFrame()
    for df in csr:
        glob = glob.append(df)
    csr = glob

In [ ]:
dd = csr[['Participant', 'Sector', 'Country', 'Year', 'Id', 'code', "English"]]

In [ ]:
csr = dd.merge(hdi[['Country', "HDI"]], on = "Country", how = "left")

In [ ]:
def sids_merg(row):
    sids_sign = None
    if row["Country"] in sids:
        sids_sign = "SIDS"
    elif (row["Country"] in tier1_countries_eu) or (row["code"] in tier1_countries):
        sids_sign = "Tier 1"
    elif row["code"] in tier2_countries:
        sids_sign = "Tier 2"
    return sids_sign
csr["SIDS"] = csr.apply(lambda x: sids_merg(x), axis = 1)

In [ ]:
csr[csr["SIDS"].isna()].Country.unique()

In [ ]:
def who_seatch(x):
    region = None
    for k, v in WHO.items():
        if x in v:
            region = k 
    if not region:
        print(x)
    return region
csr["WHO"] = csr.Country.apply(who_seatch)

In [ ]:
def agg_func(df, key = "intersection"):
    agg_inter = pd.DataFrame(df.groupby("Id").count().key_word).reset_index()
    agg_inter["Id"] = agg_inter["Id"].astype("int")
    agg_inter.rename(columns = {"key_word": key}, inplace = True)
    return agg_inter

In [ ]:
csr = csr[csr.Year > 2010]
print(csr.shape)
reference = pd.merge(csr[["Participant",'Sector', 'Year', 'Id', "code", "Country", "WHO", "HDI", "SIDS"]], agg_func(df_intersection, "intersection"), on = "Id", how = "left")
reference = pd.merge(reference, agg_func(df_health, "health"), on = "Id", how = "left")
reference = pd.merge(reference, agg_func(df_climate, "climate"), on = "Id", how = "left")
reference = pd.merge(reference, agg_func(df_gender, "gender"), on = "Id", how = "left")
reference = pd.merge(reference, agg_func(df_covid, "covid"), on = "Id", how = "left")
reference = reference.fillna(0)
#reference["n_company_year"] = reference.groupby(["Year"]).Id.transform("nunique")
reference["total_texts"] = reference.groupby(["Year", "Sector"]).Id.transform("nunique")

In [ ]:
language = pd.read_csv("./globalmulti/nonEU_langlist.csv")
language = language[["Id", "lang"]]
language_EU = pd.read_csv("./EU_lang.csv")
language = language.append(language_EU)
lang = pd.merge(csr[["Id", "English"]], language, on = "Id", how = "left")
lang = lang.fillna("en")
lang.loc[ lang[(lang.English == 0) & (lang.lang == "en")].index, "lang"] = "en-mixed"
lang.groupby("lang").count().English.sort_values()

In [ ]:
reference["intersection_texts"] = reference.groupby(["Year", "Sector", "Participant"]).intersection.transform("sum") > 0
reference["climate_texts"] = reference.groupby(["Year", "Sector", "Participant"]).climate.transform("sum") > 0
reference["health_texts"] = reference.groupby(["Year", "Sector", "Participant"]).health.transform("sum") > 0

In [ ]:
avg_sector = reference[reference.Year == 2021].groupby("Sector").agg({"intersection":"mean", "health":"mean", "climate":"mean",}).reset_index()
avg_sector = avg_sector.round(2).sort_values("Sector")

In [ ]:
sum_1 = reference.agg({"Participant": ["count", "nunique",],}).T.reset_index(drop = True)
sum_2 = reference.agg({ "health_texts": ["sum"], "climate_texts": "sum", "intersection_texts": "sum", }).reset_index(drop = True)

In [ ]:
summary = pd.concat([sum_1, sum_2, pd.DataFrame(sum_2.squeeze()/sum_1["count"].values).T], axis=1, )
summary = summary.round({"health_texts":2,"climate_texts": 2, "intersection_texts": 3})
summary.set_axis(["Companies (N)", "Companies (Unique)", "Health, (N)", "Climate, (N)", "Intersection, (N)", 
                  "Health, %", "Climate, %", "Intersection, %"], axis=1, inplace=True)

In [ ]:
# 0_0_data_summary
summary.to_csv(path + '0_0_data_summary.csv', index = False)
summary

In [ ]:
sum_1 = reference.groupby(["Year"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "Companies (N)"], axis=1, inplace=True)
sum_2 = reference.groupby(["Year"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", }).reset_index(drop = True)
sum_2.set_axis(["Climate (N)", "Health (N)", "Intersection (N)"], axis=1, inplace=True)
sum_3 = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Companies (N)"]).round(2))
sum_3.set_axis(["Climate (Prop)", "Health (Prop)", "Intersection (Prop)"], axis=1, inplace=True)
yearly_breakdown = pd.concat([sum_1, sum_2, sum_3], axis=1, )
yearly_breakdown.to_csv(path + '0_1_yearly_breakdown.csv', index = False)
yearly_breakdown

In [ ]:
reference_health = reference[reference.Sector == "Health Care Equipment & Ser..."]
sum_1 = reference_health.groupby(["Year"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "Companies (N)"], axis=1, inplace=True)
sum_2 = reference_health.groupby(["Year"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", }).reset_index(drop = True)
sum_2.set_axis(["Climate (N)", "Health (N)", "Intersection (N)"], axis=1, inplace=True)
sum_3 = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Companies (N)"]).round(2))
sum_3.set_axis(["Climate (Prop)", "Health (Prop)", "Intersection (Prop)"], axis=1, inplace=True)
yearly_breakdown_health = pd.concat([sum_1, sum_2, sum_3], axis=1, )

In [ ]:
# not company check but document check
sum_1 = reference.groupby(["Year", "WHO"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "WHO", "Total documents"], axis=1, inplace=True)
sum_2 = reference.groupby(["Year", "WHO"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
who_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
who_prop = who_prop[who_prop.WHO != 0].sort_values(["WHO", "Year"])

In [ ]:
sum_1 = reference_health.groupby(["Year", "WHO"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "WHO", "Total documents"], axis=1, inplace=True)
sum_2 = reference_health.groupby(["Year", "WHO"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
who_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
who_prop_health = who_prop_health[who_prop_health.WHO != 0].sort_values(["WHO", "Year"])

In [ ]:
# not company check but document check
sum_1 = reference.groupby(["Year", "SIDS"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "SIDS", "Total documents"], axis=1, inplace=True)
sum_2 = reference.groupby(["Year", "SIDS"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
sids_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
sids_prop = sids_prop[sids_prop.SIDS != 0].sort_values(["SIDS", "Year"])

In [ ]:
# not company check but document check
sum_1 = reference_health.groupby(["Year", "SIDS"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "SIDS", "Total documents"], axis=1, inplace=True)
sum_2 = reference_health.groupby(["Year", "SIDS"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
sids_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
sids_prop_health = sids_prop_health[sids_prop_health.SIDS != 0].sort_values(["SIDS", "Year"])

In [ ]:
# not company check but document check
sum_1 = reference.groupby(["Year", "HDI"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "HDI", "Total documents"], axis=1, inplace=True)
sum_2 = reference.groupby(["Year", "HDI"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
hdi_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
hdi_prop = hdi_prop[hdi_prop.HDI != 0].sort_values(["HDI", "Year"])

In [ ]:
sum_1 = reference_health.groupby(["Year", "HDI"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Year", "HDI", "Total documents"], axis=1, inplace=True)
sum_2 = reference_health.groupby(["Year", "HDI"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
hdi_prop_health = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
hdi_prop_health = hdi_prop_health[hdi_prop_health.HDI != 0].sort_values(["HDI", "Year"])

In [ ]:
sum_1 = reference.groupby(["Sector"]).agg({"Participant": "count",}).reset_index()
sum_1.set_axis(["Sector", "Total documents"], axis=1, inplace=True)
sum_2 = reference.groupby(["Sector"]).agg({ "climate_texts": "sum","health_texts": ["sum"], "intersection_texts": "sum", })
sum_2.set_axis(["Climate", "Health", "Intersection"], axis=1, inplace=True)
sector_prop = pd.DataFrame(sum_2.apply(lambda x: x/sum_1["Total documents"].values).round(2)).reset_index()
sector_prop = sector_prop.sort_values(["Sector"])

In [ ]:
reference_df_sector = reference.groupby(["Sector"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", }).reset_index()
reference_who = reference.groupby(["WHO", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", })
reference_who["sum"] = reference_who["intersection"] + reference_who["climate"] + reference_who["health"]
reference_who.reset_index(inplace = True)
reference_hdi = reference.groupby(["HDI", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", }).reset_index()
reference_hdi.HDI = pd.Categorical(reference_hdi.HDI, categories=['LOW HUMAN DEVELOPMENT',
       'MEDIUM HUMAN DEVELOPMENT','HIGH HUMAN DEVELOPMENT',  'VERY HIGH HUMAN DEVELOPMENT'], ordered=True)
reference_hdi.dropna(inplace = True)

reference_hdi_health = reference_health.groupby(["HDI", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", }).reset_index()
reference_hdi_health.HDI = pd.Categorical(reference_hdi_health.HDI, categories=['LOW HUMAN DEVELOPMENT',
       'MEDIUM HUMAN DEVELOPMENT','HIGH HUMAN DEVELOPMENT',  'VERY HIGH HUMAN DEVELOPMENT'], ordered=True)
reference_hdi_health.dropna(inplace = True)

reference_sids = reference.groupby(["SIDS", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", }).reset_index()
reference_sids.SIDS = pd.Categorical(reference_sids.SIDS, categories=['SIDS',
       'Tier 1','Tier 2',], ordered=True)
reference_sids.dropna(inplace = True)

reference_sids_health = reference_health.groupby(["SIDS", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", }).reset_index()
reference_sids_health.SIDS = pd.Categorical(reference_sids_health.SIDS, categories=['SIDS',
       'Tier 1','Tier 2',], ordered=True)
reference_sids_health.dropna(inplace = True)

In [ ]:
yearly_breakdown[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]].to_csv(path+ "1a_prop_of_companies.csv")
plt = pd.melt(yearly_breakdown[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]], id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)", "Climate (Prop)", "Health (Prop)"], ordered=True)
a1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2013, y = 0.85,# 0.8,  
         label = "Health", color = "#619cff") + \
annotate("text", x = 2014, y = 0.5,#0.4,
         label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2012, y = 0.3, 
         label = "Intersection", color = "red") + \
scale_y_continuous(labels = percent_format(), limits = (0, 1)) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
a1.save(path +'1a_prop_of_companies.png')
a1.save(path +'1a_prop_of_companies.pdf')
a1

In [ ]:
yearly_breakdown_health[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]].to_csv(path+ "2p_prop_of_companies.csv")
plt = pd.melt(yearly_breakdown_health[["Year","Climate (Prop)", "Health (Prop)", "Intersection (Prop)"]], id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)", "Climate (Prop)", "Health (Prop)"], ordered=True)
p2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2013, y = 0.75,# 0.8,  
         label = "Health", color = "#619cff") + \
annotate("text", x = 2014, y = 0.6,#0.4,
         label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2012, y = 0.2, 
         label = "Intersection", color = "red") + \
scale_y_continuous(labels = percent_format(), limits = (0, 1)) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
p2.save(path +'2p_prop_of_companies.png')
p2.save(path +'2p_prop_of_companies.pdf')
p2

In [ ]:
# breaks = (20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000)
b1csv = reference.groupby("Year").agg({"intersection":"sum", "health":"sum", "climate":"sum"}).reset_index()
b1csv.to_csv(path+ "1b_number_of_references.csv")
plt = pd.melt(b1csv, id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
b1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2013, y = 40000, label = "Health", color = "#619cff") + \
annotate("text", x = 2014, y = 20000, label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2012, y = 5000, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
b1.save(path +'1b_number_of_references.png')
b1.save(path +'1b_number_of_references.pdf')
b1

In [ ]:
q2csv = reference_health.groupby("Year").agg({"intersection":"sum", "health":"sum", "climate":"sum"}).reset_index()
q2csv.to_csv(path+ "2q_number_of_references.csv")
plt = pd.melt(q2csv, id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
q2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2015, y = 1900, label = "Health", color = "#619cff") + \
annotate("text", x = 2017, y = 700, label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2014, y = 500, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
# 1b_number_of_references
q2.save(path +'2q_number_of_references.png')
q2.save(path +'2q_number_of_references.pdf')
q2

In [ ]:
c1csv = reference.groupby("Year").agg({"intersection":"sum"}).reset_index()
c1csv.to_csv(path+ "1c_number_of_references_intersection.csv")
plt = pd.melt(c1csv, id_vars=['Year'] )
c1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
annotate("text", x = 2012, y = 2000, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None),) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red",)) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
c1.save(path +'1c_number_of_references_intersection.png')
c1.save(path +'1c_number_of_references_intersection.pdf')
c1

In [ ]:
r2csv = reference_health.groupby("Year").agg({"intersection":"sum"}).reset_index()
r2csv.to_csv(path+ "2r_number_of_references_intersection.csv")
plt = pd.melt(r2csv, id_vars=['Year'] )
r2 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
annotate("text", x = 2012, y = 50, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None),) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red",)) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
r2.save(path +'2r_number_of_references_intersection.png')
r2.save(path +'2r_number_of_references_intersection.pdf')
r2

In [ ]:
yearly_breakdown[["Year", "Intersection (Prop)"]].to_csv(path+ "1d_proportion_intersection.csv")
plt = pd.melt(yearly_breakdown[["Year", "Intersection (Prop)"]], id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["Intersection (Prop)",], ordered=True)
d1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
annotate("text", x = 2012, y = 0.1, label = "Intersection", color = "red") + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red",  )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
d1.save(path +'1d_proportion_intersection.png')
d1.save(path +'1d_proportion_intersection.pdf')
d1

In [ ]:
e1csv = reference.groupby("Year").agg({"intersection":"mean", "health":"mean", "climate":"mean"}).reset_index()
e1csv.to_csv(path+ "1e_avg_references.csv")
plt = pd.melt(e1csv, id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
e1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2013, y = 17, label = "Health", color = "#619cff") + \
annotate("text", x = 2014, y = 10, label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2012, y = 2, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Average number of references") 
e1.save(path +'1e_avg_references.png')
e1.save(path +'1e_avg_references.pdf')
e1

In [ ]:
e1csv = reference_health.groupby("Year").agg({"intersection":"mean", "health":"mean", "climate":"mean"}).reset_index()
e1csv.to_csv(path+ "2s_avg_references.csv")
plt = pd.melt(e1csv, id_vars=['Year'] )
plt.variable = pd.Categorical(plt.variable, categories=["intersection", "climate", "health"], ordered=True)
e1 = ggplot(plt, aes(x = "Year", y = "value", color = "variable")) + \
geom_path(aes(linetype = "variable")) + \
scale_linetype_manual(values = ("solid", 'dashdot', "--")) + \
annotate("text", x = 2013, y = 40, label = "Health", color = "#619cff") + \
annotate("text", x = 2014, y = 10, label = "Climate Change", color = "darkgreen") + \
annotate("text", x = 2012, y = 5, label = "Intersection", color = "red") + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "none") + \
scale_colour_manual(values=("red", "darkgreen", "#619cff" )) +\
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Average number of references") 
e1.save(path +'2s_avg_references.png')
e1.save(path +'2s_avg_references.pdf')
e1

In [ ]:
reference_who[reference_who.WHO !=0].to_csv(path+ "1f_who_number_references.csv")
f1 = ggplot(reference_who[reference_who.WHO !=0], aes(x = "Year", y = "intersection", color = "WHO")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
f1.save(path +'1f_who_number_references.png')
f1.save(path +'1f_who_number_references.pdf')
f1

In [ ]:
reference_who_health = reference_health.groupby(["WHO", "Year"]).agg({"intersection": "sum",
                                           "climate": "sum",
                                           "health":"sum", })
reference_who_health["sum"] = reference_who_health["intersection"] + reference_who_health["climate"] + reference_who_health["health"]
reference_who_health.reset_index(inplace = True)

In [ ]:
reference_who_health["Sector"] = "Health Care Equipment & Ser..."
reference_who_health[reference_who_health.WHO !=0].to_csv(path+ "2t_who_number_references.csv")
f1 = ggplot(reference_who_health[reference_who_health.WHO !=0], aes(x = "Year", y = "intersection", color = "WHO")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references") 
f1.save(path +'2t_who_number_references.png')
f1.save(path +'2t_who_number_references.pdf')
f1

In [ ]:
who_prop[who_prop.WHO !=0].to_csv(path+ "1g_who_prop_references.csv")
g1 = ggplot(who_prop[who_prop.WHO !=0], aes(x = "Year", y = "Intersection", color = "WHO")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
g1.save(path +'1g_who_prop_references.png')
g1.save(path +'1g_who_prop_references.pdf')
g1

In [ ]:
who_prop_health[who_prop_health.WHO !=0].to_csv(path+ "2u_who_prop_references.csv")
g1 = ggplot(who_prop_health[who_prop_health.WHO !=0], aes(x = "Year", y = "Intersection", color = "WHO")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
g1.save(path +'2u_who_prop_references.png')
g1.save(path +'2u_who_prop_references.pdf')
g1

In [ ]:
#1h_sids_number_references.csv
reference_sids["sum"] = reference_sids["intersection"] + reference_sids["climate"] + reference_sids["health"] 
reference_sids.to_csv(path+ "1h_sids_number_references.csv")
h1 = ggplot(reference_sids[reference_sids.SIDS != 0], aes(x = "Year", y = "intersection", color = "SIDS")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references")
h1.save(path +'1h_sids_number_references.png')
h1.save(path +'1h_sids_number_references.pdf')
h1

In [ ]:
#2v_sids_number_references.csv
reference_sids_health["sum"] = reference_sids_health["intersection"] + reference_sids_health["climate"] + reference_sids_health["health"] 
reference_sids.to_csv(path+ "2v_sids_number_references.csv")
h1 = ggplot(reference_sids_health[reference_sids_health.SIDS != 0], aes(x = "Year", y = "intersection", color = "SIDS")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references")
h1.save(path +'2v_sids_number_references.png')
h1.save(path +'2v_sids_number_references.pdf')
h1

In [ ]:
sids_prop[sids_prop.SIDS !=0].to_csv(path+ "1i_sids_prop_references.csv")
i1 = ggplot(sids_prop[sids_prop.SIDS !=0], aes(x = "Year", y = "Intersection", color = "SIDS")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
i1.save(path +'1i_sids_prop_references.png')
i1.save(path +'1i_sids_prop_references.pdf')
i1

In [ ]:
sids_prop_health[sids_prop_health.SIDS !=0].to_csv(path+ "2w_sids_prop_references.csv")
i1 = ggplot(sids_prop_health[sids_prop_health.SIDS !=0], aes(x = "Year", y = "Intersection", color = "SIDS")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
i1.save(path +'2w_sids_prop_references.png')
i1.save(path +'2w_sids_prop_references.pdf')
i1

In [ ]:
reference_hdi["sum"] = reference_hdi["intersection"] + reference_hdi["climate"] + reference_hdi["health"] 
reference_hdi.to_csv(path+ "1j_hdi_number_references.csv")
j1 = ggplot(reference_hdi[reference_hdi.HDI != 0], aes(x = "Year", y = "intersection", color = "HDI")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references")
j1.save(path +'1j_hdi_number_references.png')
j1.save(path +'1j_hdi_number_references.pdf')
j1

In [ ]:
reference_hdi_health["sum"] = reference_hdi_health["intersection"] + reference_hdi_health["climate"] + reference_hdi_health["health"] 
reference_hdi_health["Sector"] = "Health Care Equipment & Ser..."
reference_hdi_health.to_csv(path+ "2x_hdi_number_references.csv")
j1 = ggplot(reference_hdi_health[reference_hdi_health.HDI != 0], aes(x = "Year", y = "intersection", color = "HDI")) + \
geom_path(aes()) + \
scale_y_continuous(limits = (0, None), ) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Total number of references")
j1.save(path +'2x_hdi_number_references.png')
j1.save(path +'2x_hdi_number_references.pdf')
j1

In [ ]:
hdi_prop[hdi_prop.HDI !=0].to_csv(path+ "1k_hdi_prop_references.csv")
k1 = ggplot(hdi_prop[hdi_prop.HDI !=0], aes(x = "Year", y = "Intersection", color = "HDI")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, 0.5)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
# 1k_hdi_prop_references

k1.save(path +'1k_hdi_prop_references.png')
k1.save(path +'1k_hdi_prop_references.pdf')
k1

In [ ]:
hdi_prop_health[hdi_prop_health.HDI !=0].to_csv(path+ "2y_hdi_prop_references.csv")
k1 = ggplot(hdi_prop_health[hdi_prop_health.HDI !=0], aes(x = "Year", y = "Intersection", color = "HDI")) + \
geom_path(aes()) + \
scale_y_continuous(labels = percent_format(), limits = (0, None)) + \
theme_minimal() + \
theme(legend_position = "right") + \
scale_x_continuous(breaks=(2012,2014,2016,2018, 2020), ) +\
labs(y = "Proportion of companies, %") 
# 1k_hdi_prop_references

k1.save(path +'2y_hdi_prop_references.png')
k1.save(path +'2y_hdi_prop_references.pdf')
k1

In [ ]:
# 1l_sector_references_2021
reference_df_sector.to_csv(path+"1l_sector_references_2021.csv")
reference_df_sector

In [ ]:
# 1m_avg_sector_references_2020.csv
avg_sector.to_csv(path + "1m_avg_sector_references_2021.csv")
avg_sector

In [ ]:
# 1n_prop_companies_sector.csv  2021
sector_prop["Year"] = 2021
sector_prop.to_csv(path + "1n_prop_companies_sector.csv")
melt = sector_prop.melt(id_vars=["Sector", 'Year'])
melt.variable = pd.Categorical(melt.variable, categories=["Intersection", "Climate", "Health"], ordered=True)
n1 = ggplot(melt, aes(x = "Sector", y = "value", fill = "variable")) + \
  geom_bar(stat = "identity", position = "dodge") + \
  theme_minimal() + \
  scale_y_continuous(labels = percent_format()) + \
  theme(axis_text_x = element_text(angle = 90, vjust = 1, hjust=0.5, size = 10 ), ) + \
  theme(legend_position = (0.5, -0.4), legend_direction="horizontal", figure_size=(14, 7)) + \
  scale_fill_manual(values=("red", "darkgreen", "#619cff" )) +\
  labs(fill = "",
       y = "Proportion of companies, %",
       x = "\nSector")
n1.save(path +'1n_prop_companies_sector.png')
n1.save(path +'1n_prop_companies_sector.pdf')
n1

In [ ]:
reference["covid_texts"] = reference.groupby(["Year", "Sector", "Id"]).covid.transform("sum") > 0
reference["gender_texts"] = reference.groupby(["Year", "Sector", "Id"]).gender.transform("sum") > 0

In [ ]:
covid_inter = reference[(reference.Year > 2019) & (reference.intersection > 0)].groupby(["Year"]).agg({"Id": "nunique", "covid":"sum", "covid_texts": "sum",})
covid_inter["prop"] = (covid_inter["covid"]/covid_inter["Id"]).round(2)
covid_inter.set_axis(["total_docs", "hits", "documents", "prop_doct"], axis=1, inplace=True)
covid_inter["hits"] = covid_inter["hits"].astype("int")
covid_inter.reset_index(inplace = True)
covid_inter.to_csv(path + "1o_intersection_covid.csv")
covid_inter

In [ ]:
o1 = ggplot(covid_inter, aes(x = "Year", y = "prop_doct", group = 1)) +\
  geom_line(color = "#cc0055") +\
  theme_minimal() +\
  labs(x= "Year",
       y = "Proportion of documents\n Covid mention in intersection, %\n") +\
  scale_y_continuous(labels = percent_format(), limits = (0, None))
o1.save(path +'1o_prop_intersecion_covid.png')
o1.save(path +'1o_prop_intersecion_covid.pdf')
o1

In [ ]:
# 1o_intersection_gender
gender_inter = reference[(reference.Year > 2010) & (reference.intersection > 0)].groupby(["Year"]).agg({"Id": "nunique", "gender":"sum", "gender_texts": "sum", })
gender_inter["prop"] = (gender_inter["gender"]/gender_inter["Id"]).round(2)
gender_inter.set_axis(["total_docs", "hits", "documents", "prop_doct"], axis=1, inplace=True)
gender_inter["hits"] = gender_inter["hits"].astype("int")
gender_inter.reset_index(inplace = True)

In [ ]:
gender_inter.to_csv(path + "1o_intersection_gender.csv", index = False)
o1 = ggplot(gender_inter, aes(x = "Year", y = "prop_doct", group = 1)) +\
  geom_line(color = "#cc0055") +\
  theme_minimal() +\
  labs(x= "Year",
       y = "Proportion of documents\n Gender mention in intersection, %\n") +\
  scale_y_continuous(labels = percent_format(), limits = (0, None))
o1.save(path +'1o_prop_intersecion_gender.png')
o1.save(path +'1o_prop_intersecion_gender.pdf')
o1